# --- Day 15: Science for Hungry People ---

Today, you set out on the task of perfecting your milk-dunking cookie recipe. All you have to do is find the right balance of ingredients.

Your recipe leaves room for exactly 100 teaspoons of ingredients. You make a list of the remaining ingredients you could use to finish the recipe (your puzzle input) and their properties per teaspoon:

- capacity (how well it helps the cookie absorb milk)
- durability (how well it keeps the cookie intact when full of milk)
- flavor (how tasty it makes the cookie)
- texture (how it improves the feel of the cookie)
- calories (how many calories it adds to the cookie)

You can only measure ingredients in whole-teaspoon amounts accurately, and you have to be accurate so you can reproduce your results in the future. The total score of a cookie can be found by adding up each of the properties (negative totals become 0) and then multiplying together everything except calories.

For instance, suppose you have these two ingredients:

- Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8
- Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3

Then, choosing to use 44 teaspoons of butterscotch and 56 teaspoons of cinnamon (because the amounts of each ingredient must add up to 100) would result in a cookie with the following properties:

- A capacity of 44x-1 + 56x2 = 68
- A durability of 44x-2 + 56x3 = 80
- A flavor of 44x6 + 56*x2 = 152
- A texture of 44x3 + 56x-1 = 76

Multiplying these together (68 * 80 * 152 * 76, ignoring calories for now) results in a total score of 62842880, which happens to be the best score possible given these ingredients. If any properties had produced a negative total, it would have instead become zero, causing the whole score to multiply to zero.

Given the ingredients in your kitchen and their properties, what is the total score of the highest-scoring cookie you can make?

In [7]:
# Import pandas and read in the input file, saved as a .csv
import pandas as pd
import re

# Read in original input, then create new dataframe (data) with only the relevent information
df = pd.read_csv(r'C:\Users\jakja\Documents\Python Scripts\Day 15 Input.csv')
inpt = []
for row in df["Ingredient"]:
    inpt += [row.split(' ')]
data = pd.DataFrame()
for line in inpt:
    data = data.append({'Name':line[0], 'Capacity':int(re.sub("[^\d\-]", "", line[2])), 'Durability':int(re.sub("[^\d\-]", "", line[4])), 'Flavor':int(re.sub("[^\d\-]", "", line[6])), 'Texture':int(re.sub("[^\d\-]", "", line[8])), 'Calories':int(re.sub("[^\d\-]", "", line[10]))}, ignore_index=True)

# Create a list of ingredients
ingredients = data['Name'].tolist() 
print(ingredients)

# Make an empty dictionary for the recipe
recipe = {}
for row in data['Name']:
    recipe[row] = 0
print(recipe)

# Make 'Name' the index of the dataframe
data.set_index('Name', inplace=True, drop=True)
print(data)

['Sprinkles:', 'Butterscotch:', 'Chocolate:', 'Candy:']
{'Sprinkles:': 0, 'Butterscotch:': 0, 'Chocolate:': 0, 'Candy:': 0}
               Calories  Capacity  Durability  Flavor  Texture
Name                                                          
Sprinkles:          3.0       2.0         0.0    -2.0      0.0
Butterscotch:       3.0       0.0         5.0    -3.0      0.0
Chocolate:          8.0       0.0         0.0     5.0     -1.0
Candy:              8.0       0.0        -1.0     0.0      5.0


In [8]:
# Make a function to assign the teaspoons in every combo and test the score. 
# This function will define a for loop for each ingredient, by index, to count through all possible quantities
# The function will then call itself (possibly bad practice, should investigate) to assess the remaining ingredients
# Using this method, it will assess every possible combination once
# This script is written to optimise for any number of ingredients, with the 5 attributes fixed
# When originally written for a fixed number of ingredients, script was faster (see comments when calculating attributes)

# The function will need to be passed the index of the ingredient it is examining, the current recipe and the current topScore
def ingred(index, recipe, topScore):

    # Initiate a variable for the amount of space left in the recipe, starting at total (100) and subtracting the amount allocated to previous ingredients
    spaceLeft = totalTeaspoons
    for x in range(0, index):
        spaceLeft -= recipe.get(ingredients[x])

    # If the ingredient being examined is not the last in the list, you will need to go through the function again
    if index <= len(recipe) - 2:
        
        # Use a for loop to loop through all possible quantities of the ingredient, based on the remaining space
        for n in range(0, spaceLeft + 1):                 
            recipe[ingredients[index]] = n
            
            # Increase the index to look at the next ingredient, then run this function again on that ingredient
            index += 1           
            index, recipe, topScore = ingred(index, recipe, topScore)
    
    # If the last ingredient is being examined, then the total calculation will follow
    else:     
        
        # Allocate the remaining space in the recipe to the final ingredient
        recipe[ingredients[index]] = spaceLeft

        # Calculate the 4 attributes of the cookie 
        # This is written to work on any number of ingredients, but much faster to hard code the equations if ingredients are fixed
        capacity = 0
        for x in range(len(ingredients)):
            capacity += data.at[ingredients[x], 'Capacity'] * recipe[ingredients[x]]
        durability = 0
        for x in range(len(ingredients)):
            durability += data.at[ingredients[x], 'Durability'] * recipe[ingredients[x]] 
        flavor = 0
        for x in range(len(ingredients)):
            flavor += data.at[ingredients[x], 'Flavor'] * recipe[ingredients[x]]
        texture = 0
        for x in range(len(ingredients)):
            texture += data.at[ingredients[x], 'Texture'] * recipe[ingredients[x]]  
        
        # Check if any attributes are negative, in which case set the score to 0, otherwise set the score to the attribute product
        if capacity <= -1 or durability <= -1 or flavor <= -1 or texture <= -1:
            score = 0
        else:
            score = capacity * durability * flavor * texture
        
        # Check if this cookie's score is greater than the current top score. If so, overwrite the top score.
        if score >= topScore + 1:
            # print('New Top Score: ' + str(score))
            # print('Recipe: ' + str(recipe))
            topScore = score
    
    # Reduce the index to go back to the previous ingredient and the next step in it's for loop
    index -= 1
    
    # Print out a message each time the initial for loop progresses (just to monitor progress)
    # if index == 0:    
        # print('Assessing ', ingredients[index], ' with value ', recipe.get(ingredients[0]) + 1)
    
    # Return the same outputs as the inputs (so the function can feed itself)
    return index, recipe, topScore     

In [9]:
# Set recipe size and run the function, with an initial index of 0, the recipe as defined and an initial Top Score of 0
totalTeaspoons = 100
print('\nThe top cookie scored ', ingred(0, recipe, 0)[2])


The top cookie scored  21367368.0


# --- Part Two ---

Your cookie recipe becomes wildly popular! Someone asks if you can make another recipe that has exactly 500 calories per cookie (so they can use it as a meal replacement). Keep the rest of your award-winning process the same (100 teaspoons, same ingredients, same scoring system).

For example, given the ingredients above, if you had instead selected 40 teaspoons of butterscotch and 60 teaspoons of cinnamon (which still adds to 100), the total calorie count would be 40*8 + 60*3 = 500. The total score would go down, though: only 57600000, the best you can do in such trying circumstances.

Given the ingredients in your kitchen and their properties, what is the total score of the highest-scoring cookie you can make with a calorie total of 500?

In [275]:
# Set up is the same as for Part 1
    
# Function method is the same for Part 1, this time entitles 'calor'
def calor(index, recipe, topScore):

    spaceLeft = totalTeaspoons
    for x in range(0, index):
        spaceLeft -= recipe.get(ingredients[x])

    if index <= len(recipe) - 2:
       
        for n in range(0, spaceLeft + 1):                 
            recipe[ingredients[index]] = n
            
            index += 1
            index, recipe, topScore = calor(index, recipe, topScore)                      
            
    else:     
        
        recipe[ingredients[index]] = spaceLeft

        capacity = 0
        for x in range(len(ingredients)):
            capacity += data.at[ingredients[x], 'Capacity'] * recipe[ingredients[x]]
        durability = 0
        for x in range(len(ingredients)):
            durability += data.at[ingredients[x], 'Durability'] * recipe[ingredients[x]] 
        flavor = 0
        for x in range(len(ingredients)):
            flavor += data.at[ingredients[x], 'Flavor'] * recipe[ingredients[x]]
        texture = 0
        for x in range(len(ingredients)):
            texture += data.at[ingredients[x], 'Texture'] * recipe[ingredients[x]] 
        # Add new attribute, calories
        calories = 0
        for x in range(len(ingredients)):
            calories += data.at[ingredients[x], 'Calories'] * recipe[ingredients[x]] 
        
        if capacity <= -1 or durability <= -1 or flavor <= -1 or texture <= -1:
            score = 0
        
        # Add new constraint, that calories must equal 500, or score is 0
        elif calories != 500:
            score = 0
            
        else:
            score = capacity * durability * flavor * texture
        
        if score >= topScore + 1:
            # print('New Top Score: ' + str(score))
            # print('Recipe: ' + str(recipe))
            topScore = score
        
    index -= 1
    # if index == 0:    
        # print('Assessing ', ingredients[index], ' with value ', recipe.get(ingredients[0]))
            
    return index, recipe, topScore      

In [276]:
# Set recipe size and run the function, with an initial index of 0, the recipe as defined and an initial Top Score of 0
totalTeaspoons = 100
print('\nThe top cookie scored ', calor(0, recipe, 0)[2])


The top cookie scored  1766400.0
